<h2><center>Anisotropic Absorption with Delta Radiance Distribution in 2D</center></h2>

This notebook explores the numerical solution of the anisotropic $P_1$ Approximation, when only absorbing media is present and when using a Delta distribution as closure (see survey chapter 2.2.2) in 2-dimensional domain. We will place a unit point light at the center of the domain and compare the solution against groundtruth results.

---------------------------
Let us consider the diffusion equation for anisotropic media, when being exposed to a delta distribution for the second moment of the radiance field (see section 2.2.2 in the survey):

$$\nabla\cdot\left[\left(M^{-1}\frac{\nabla\phi}{\lVert M^{-1}\nabla\phi \rVert}\right)\phi\right] = \mu_0\left[-\sigma_t + \sigma_s\right]\phi + Q_0$$

In 2d, $M$ is given as:

$$M=\left(-\mu_2\left[\sigma_t\right] + \mu_1\left[\sigma_s\mu_1\left[f_p\right]\right]\right) $$

Note that we define the moments like this (which explains the missing $\frac{1}{2\pi}$ and $\frac{2}{2\pi}$ factors from the equations in the report):

$$
\begin{align}
\mu_0\left[L\right] &= \frac{1}{2\pi}\int_\Omega{L\mathrm{dt}\omega}\\
\mu_2\left[L\right] &= \frac{2}{2\pi}\int_\Omega{L\omega_i\omega_j\mathrm{dt}\omega}
\end{align}
$$

In a purely absorbing media, $\sigma_s=0$ and therefore the equations above reduce to:

$$
\nabla\cdot\left[\left(M^{-1}\frac{\nabla\phi}{\lVert M^{-1}\nabla\phi \rVert}\right)\phi\right] = -\mu_0\left[\sigma_t\right]\phi + Q_0
$$

and

$$M=-\mu_2\left[\sigma_t\right]$$

---------------------------
If $\sigma_t$ is isotropic, its moments in 2d are:

$$
\mu_0[\sigma_t] = \sigma_t
$$

$$
\mu_2[\sigma_t] = 0\mathrm{I}
$$

and the anisotropic equations above reduce to the isotropic solution:

$$M^{-1}=-\infty$$

$$
\nabla\cdot\left[-\frac{\nabla\phi}{\lVert\nabla\phi\rVert}\phi\right] = -\sigma_t\phi + Q_0
$$

---------------------------
Since the diffusion coefficient is a matrix, the discretization of the anisotropic $P_1$ equation for a delta radiance distribution is quite involved. Please see the notebook "anisotropic_absorption_2d_discretization" for all the details.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import numpy as np
import matplotlib.pyplot as plt
from util import *
import solver as solver

In [2]:
# mu0_sigma_t: zero moment of the extinction coefficient at cell centers
def run_sim( domain, numIterations, mu0_sigma_t, Q, M_inv_x, M_inv_y, phi_boundary, phi_groundtruth_grid, debug = False ):
    # setup/reset grids ---
    phi = np.zeros((domain.res, domain.res)) # scalar field for which we want to solve 
    np.copyto(phi, phi_groundtruth_grid)


    # solve ---
    for step in range(numIterations):
        # call inner loop in c++
        #py::array_t<double> phi_array, // phi grid
        #py::array_t<double> Q_array, // grid of emission values
        #py::array_t<double> M_x_array, // grid of 2x2 matrices which represent anisotropy on faces perpendicular to x direction
        #py::array_t<double> M_y_array, // grid of 2x2 matrices which represent anisotropy on faces perpendicular to y direction
        #py::array_t<double> phi_boundary_array, // boundary values for phi
        #double h // voxelsize (in x and y)
        test = solver.iterate_2d_anisotropic_absorption(phi, mu0_sigma_t, Q, M_inv_x, M_inv_y, phi_boundary, domain.h, debug)
        if len(test)>0:
            print(test)
        
    return phi

In [7]:
# SCENE: single pointlight -----------------------------------------------
# setup domain ---
res = 161
size = 1.0
numIterations = 20500
domain = Domain2D(size, res)

# we define angular dependent sigma_t, by a density constant multiplied
# with an angular distribution of projected distances (we use sggx for that)
density_constant = 0.0
frame_s = np.array([1.0, 0.0])
projectedDistances = np.array([1.0, 1.0])
sggx = SGGX2D(frame_s, projectedDistances)

# initialize field with zero moment of sigma_t at cell centers ---
# here we assume sigma_t to be homogeneous
mu0_sigma_t = sggx.get_moment_0()
mu0_sigma_t_grid = np.ones((domain.res, domain.res))*density_constant*mu0_sigma_t

# initialize emission field ---
Q_grid = np.zeros((domain.res, domain.res))
pointlight1_center_ls = np.array([0.5, 0.5])
pointlight1_center_ws = domain.localToWorld(pointlight1_center_ls)
pointlight1_voxel = domain.voxelToIndex(domain.localToVoxel(pointlight1_center_ls))
Q_grid[ pointlight1_voxel[0], pointlight1_voxel[1] ] = 1.0/(domain.h*domain.h)
print("pointlightvoxel={} {}".format(pointlight1_voxel[0], pointlight1_voxel[1]))

# initialize anisotropy matrices ---
# currently we assume a homogeneous (bit still angular dependent) sigma_t and therefore get a spatially constant M matrix
M = sggx.get_moment_2()
print("det(M)={}".format(np.linalg.det(M)))
if np.abs(np.linalg.det(M)) < 1.0e-4:
    print("Unable to invert M. Setting negative identity.")
    # We can not invert our second moment. This happens when our sigma_t is
    # isotropic (or very close) or when density becomes 0. We handle this case by setting our inverse
    # of M to being -Identity, since in this case it cancels out anyway
    #M = -np.identity(2)
    M = np.identity(2)

M_inv = np.linalg.inv(M)

print("M^(-1)=")
print(M_inv)


# In each voxel we store two matrices. One at the left and another at the bottom cell face (staggered grid).
# M_x holds all matrices on the cell faces in x direction and M_y in y direction
M_inv_x_grid = np.zeros((domain.res+1, domain.res+1, 2, 2))
M_inv_y_grid = np.zeros((domain.res+1, domain.res+1, 2, 2))
for i in range(0, domain.res+1):
    for j in range(0, domain.res+1):
        M_inv_x_grid[i, j] = M_inv
        M_inv_y_grid[i, j] = M_inv
        

def compute_lightdir(pWS):
    center = pointlight1_center_ws
    lightDir = center-pWS
    dist = np.linalg.norm(lightDir)
    if not dist>0:
        return np.zeros(2)
    return lightDir/dist
        
def compute_phi_groundtruth(pWS):
    ''' Compute groundtruth solution for single pointlight
    '''
    center = pointlight1_center_ws
    lightDir = center-pWS
    dist = np.linalg.norm(lightDir)
    if not dist>0:
        return 0.0

    lightDir = lightDir/dist
    power = 1.0

    radiance = power/(2.0*np.pi)

    L0 = radiance
    L0 *= geometry_term_2d(center, None, pWS, None)
    #sigma_t = sggx.projectedDistance(lightDir)*density_constant
    #L0 *= np.exp(-sigma_t*dist)
    
    return L0


# compute groundtruth results for validation and boundary conditions ---
#phi_groundtruth_grid = domain.rasterize(compute_phi_groundtruth)
#gradphi_groundtruth_grid = domain.gradient(phi_groundtruth_grid)
#gradphi_groundtruth_grid = domain.rasterize(compute_gradphi_groundtruth, (domain.res, domain.res, 2))

# initialize boundary values from groundtruth results ---
# we do this to see the solution without any boundary effects
#phi_boundary_grid = np.copy(phi_groundtruth_grid)

# run simuation ---
numIterations = 20500
#numIterations = 1
debug = False
#phi_grid = run_sim( domain, numIterations, mu0_sigma_t_grid, Q_grid, M_inv_x_grid, M_inv_y_grid, phi_boundary_grid, phi_groundtruth_grid, debug)
#np.savetxt("anisotropic_absorption_2d/solver.txt", phi_grid, fmt="%f")
#np.savetxt("anisotropic_absorption_2d/phi_groundtruth.txt", phi_groundtruth_grid, fmt="%f")
#np.savetxt("anisotropic_absorption_2d/gradphi_x_groundtruth.txt", gradphi_groundtruth_grid[:, :, 0], fmt="%f")
#np.savetxt("anisotropic_absorption_2d/gradphi_y_groundtruth.txt", gradphi_groundtruth_grid[:, :, 1], fmt="%f")


pointlightvoxel=80 80
det(M)=-2.908807634596448e-32
Unable to invert M. Setting negative identity.
M^(-1)=
[[ 1.  0.]
 [ 0.  1.]]


In [10]:
res_x = domain.res
res_y = domain.res
h = domain.h

def index_2d(i, j, res_x, stride = 1):
    return i*res_x*stride + j*stride;

#phi = np.zeros((domain.res, domain.res))
#np.copyto(phi, phi_groundtruth_grid)
#phi = phi.reshape((domain.res*domain.res))

#phi_at_cellfaces = np.zeros((domain.res*domain.res))
#gradaphi_x_at_cellfaces = np.zeros((domain.res*domain.res))
#gradaphi_y_at_cellfaces = np.zeros((domain.res*domain.res))
residuum = np.zeros((domain.res*domain.res))


#delta = 1.0/161.0
delta = 1.0/100000.0
delta_x = np.array([delta, 0.0])
delta_y = np.array([0.0, -delta])


def matrix_vector_prod( M, v ):
    return np.array( [M[0, 0] * v[0] + M[0, 1] * v[1], M[1, 0] * v[0] + M[1, 1] * v[1]] )

def compute_first_moment_radiance_field_groundtruth(pWS):
    phi = compute_phi_groundtruth(pWS)
    return compute_lightdir(pWS)*phi

def compute_div_first_moment_radiance_field_groundtruth(pWS):
    E_xp = compute_first_moment_radiance_field_groundtruth(pWS+delta_x)
    E_xm = compute_first_moment_radiance_field_groundtruth(pWS-delta_x)
    E_yp = compute_first_moment_radiance_field_groundtruth(pWS+delta_y)
    E_ym = compute_first_moment_radiance_field_groundtruth(pWS-delta_y)
    #return (E_xp[0]-E_xm[0])/(2.0*delta) + (E_yp[1]-E_ym[1])/(2.0*delta)
    return (E_xp[0]-E_xm[0])/(2.0*delta)
#return (E_xp[0]-E_xm[0])/(2.0*delta)


   
def compute_second_moment_radiance_field_groundtruth(pWS):
    phi = compute_phi_groundtruth(pWS)
    d = compute_lightdir(pWS)
    T = np.outer(d, d)
    return T*phi
    
def compute_div_second_moment_radiance_field_groundtruth(pWS):
    T_xp = compute_second_moment_radiance_field_groundtruth(pWS+delta_x)
    T_xm = compute_second_moment_radiance_field_groundtruth(pWS-delta_x)
    T_yp = compute_second_moment_radiance_field_groundtruth(pWS+delta_y)
    T_ym = compute_second_moment_radiance_field_groundtruth(pWS-delta_y)
    
    # see https://en.wikipedia.org/wiki/Tensor_derivative_(continuum_mechanics)#Cartesian_coordinates_2
    divT = np.zeros(2)
    divT[0] = (T_xp[0,0] - T_xm[0,0])/(2.0*delta) + (T_yp[0,1] - T_ym[0,1])/(2.0*delta)
    divT[1] = (T_xp[1,0] - T_xm[1,0])/(2.0*delta) + (T_yp[1,1] - T_ym[1,1])/(2.0*delta)
    return divT

    
def compute_gradphi_groundtruth(pWS):
    ''' Compute gradient of phi groundtruth solution for single pointlight
    '''
    grad_x = (compute_phi_groundtruth(pWS+delta_x) - compute_phi_groundtruth(pWS-delta_x))/(2.0*delta)
    grad_y = (compute_phi_groundtruth(pWS+delta_y) - compute_phi_groundtruth(pWS-delta_y))/(2.0*delta)
    return np.array([grad_x, grad_y])

def compute_Dgradphi_groundtruth(pWS):
    ''' Compute gradient of phi groundtruth solution for single pointlight
    '''    
    phi = compute_phi_groundtruth(pWS)
    gradphi = compute_gradphi_groundtruth(pWS)
    #length = np.linalg.norm(gradphi)
    length = np.linalg.norm(matrix_vector_prod(M_inv, gradphi))
    D = 0.0
    if length>0.0:
        #D = phi/length*np.diag([1, 1])
        D = M_inv*phi/length
    return matrix_vector_prod(D, gradphi)
        

def compute_residuum(pWS):
    phi = compute_phi_groundtruth(pWS)
    
    LHS = 0.0
    LHS += (compute_Dgradphi_groundtruth(pWS+delta_x)[0] - compute_Dgradphi_groundtruth(pWS-delta_x)[0])/(2.0*delta)
    LHS += (compute_Dgradphi_groundtruth(pWS+delta_y)[1] - compute_Dgradphi_groundtruth(pWS-delta_y)[1])/(2.0*delta)
    RHS = -mu0_sigma_t*density_constant*phi
    return np.abs(RHS-LHS)

#def compute_E_groundtruth(pWS):
#    divT = compute_div_second_moment_radiance_field_groundtruth(pWS)
#    return matrix_vector_prod(M_inv, divT)

def compute_residuum_test(pWS):
    phi = compute_phi_groundtruth(pWS)
    
    phi_xp = compute_phi_groundtruth(pWS+delta_x)
    phi_xm = compute_phi_groundtruth(pWS-delta_x)
    
    #divE = compute_div_first_moment_radiance_field_groundtruth(pWS)
    #LHS = divE
    RHS = 0.0 #-mu0_sigma_t*density_constant*phi
   
    #E = compute_first_moment_radiance_field_groundtruth(pWS)
    #LHS = matrix_vector_prod(M, E)
    #RHS = compute_div_second_moment_radiance_field_groundtruth(pWS)
    
    #phi = compute_phi_groundtruth(pWS)
    
    #LHS = E_xp[0]
    #RHS = -mu0_sigma_t*density_constant*phi
    #return np.abs(RHS-LHS)
    #return np.abs(divE)
    return np.abs((phi_xp-phi_xm)/(2.0*delta))
    #return LHS
    
# compute groundtruth results for validation and boundary conditions ---
residuum = domain.rasterize(compute_residuum_test, (domain.res, domain.res))

#gradphi_cellcenters = np.zeros((domain.res, domain.res, 2))
#for i in range(res_x-1):
#    for j in range(res_y-1):
#        norm = np.linalg.norm(gradphi_groundtruth_grid[i,j])
#        if norm > 0.0:
#            v = phi_groundtruth_grid[i,j]/norm*gradphi_groundtruth_grid[i,j]
#            gradphi_cellcenters[i, j, 0] = v[0]
#            gradphi_cellcenters[i, j, 1] = v[1]
        

        
for i in range(res_x-2):
    for j in range(res_y-2):
        idx = index_2d(i, j, res_x);
        idx_xp = index_2d(i, j+1, res_x);
        idx_xpyp = index_2d(i-1, j+1, res_x);
        idx_xm = index_2d(i, j-1, res_x);
        idx_xmyp = index_2d(i-1, j-1, res_x);
        idx_yp = index_2d(i-1, j, res_x);
        idx_ym = index_2d(i+1, j, res_x);
        idx_xpym = index_2d(i+1, j+1, res_x);
        idx_xmym = index_2d(i+1, j-1, res_x);

        '''
        # compute phi at cell faces ---
        phi2 = [0, 0, 0, 0];
        phi2[0] = (phi[idx] + phi[idx_xm])/2.0;
        phi2[1] = (phi[idx_xp] + phi[idx])/2.0;
        phi2[2] = (phi[idx] + phi[idx_ym])/2.0;
        phi2[3] = (phi[idx_yp] + phi[idx])/2.0;
        phi_at_cellfaces[idx] = phi2[1]
        
        # compute phi gradient at cell faces ---
        # index 0 == left face (i-1/2)
        # index 1 == right face (i+1/2)
        # index 2 == bottom face (j-1/2)
        # index 3 == top face (j+1/2)
        grad_phi = [np.zeros(2) for k in range(4)];
        grad_phi[0][0] = (phi[idx] - phi[idx_xm])/h;
        grad_phi[0][1] = (phi[idx_xmyp] + phi[idx_yp] - phi[idx_xmym] - phi[idx_ym])/(4.0*h);

        grad_phi[1][0] = (phi[idx_xp] - phi[idx])/h;
        grad_phi[1][1] = (phi[idx_yp] + phi[idx_xpyp] - phi[idx_ym] - phi[idx_xpym])/(4.0*h);

        grad_phi[2][0] = (phi[idx_xp] + phi[idx_xpym] - phi[idx_xm] - phi[idx_xmym])/(4.0*h);
        grad_phi[2][1] = (phi[idx] - phi[idx_ym])/h;

        grad_phi[3][0] = (phi[idx_xpyp] + phi[idx_xp] - phi[idx_xmyp] - phi[idx_xm])/(4.0*h);
        grad_phi[3][1] = (phi[idx_yp] - phi[idx])/h;
        
        gradaphi_x_at_cellfaces[idx] = grad_phi[1][0]
        gradaphi_y_at_cellfaces[idx] = grad_phi[1][1]
        
        # compute D at cell faces ---
        D = [np.zeros((2, 2)) for k in range(4)];
        for k in range(4):
            v = np.array( [M_inv[0, 0] * grad_phi[k][0] + M_inv[0, 1] * grad_phi[k][1], M_inv[1, 0] * grad_phi[k][0] + M_inv[1, 1] * grad_phi[k][1]] )
            #coeff = phi2[k]/np.linalg.norm(v);
            #D[k] = M_inv*coeff;
            
            coeff = phi2[k]/np.linalg.norm(grad_phi[k]);
            D[k] = coeff;
            
        # now lets check the first discretization step -------
        # compute D*gradphi vectors at cell faces
        #Dgradphi = [np.zeros(2) for i in range(4)];
        Dgradphi = [0.0 for i in range(4)];
        for k in range(4):
            #Dgradphi[k] = np.array( [D[k][0, 0] * grad_phi[k][0] + D[k][0, 1] * grad_phi[k][1], D[k][1, 0] * grad_phi[k][0] + D[k][1, 1] * grad_phi[k][1]] )
            Dgradphi[k] = D[k]
        
        #LHS = (Dgradphi[1][0] - Dgradphi[0][0])/h + (Dgradphi[3][1] - Dgradphi[2][1])/h
        #RHS = mu0_sigma_t*phi[idx] -Q_grid[i, j]
        #LHS = (Dgradphi[1] - Dgradphi[0])/h + (Dgradphi[3] - Dgradphi[2])/h
        #RHS = 0.0
        '''
        
        #LHS = (gradphi_cellcenters[i, j+1, 0]-gradphi_cellcenters[i, j-1, 0])/(2.0*h) + (gradphi_cellcenters[i-1, j, 1]-gradphi_cellcenters[i+1, j, 1])/(2.0*h)
        #RHS = 0.0
        
        
        #residuum[idx] = np.abs(RHS-LHS)
        #residuum[idx] = LHS
        pass
        
        

        
        
#np.savetxt("anisotropic_absorption_2d/discretization_phi.txt", phi_at_cellfaces.reshape((res_y, res_x)), fmt="%f")
#np.savetxt("anisotropic_absorption_2d/discretization_gradphi_x.txt", gradaphi_x_at_cellfaces.reshape((res_y, res_x)), fmt="%f")
#np.savetxt("anisotropic_absorption_2d/discretization_gradphi_y.txt", gradaphi_y_at_cellfaces.reshape((res_y, res_x)), fmt="%f")
np.savetxt("anisotropic_absorption_2d/discretization_residuum.txt", residuum.reshape((res_y, res_x)), fmt="%f")
        









In [ ]:
center_voxel = domain.voxelToIndex(domain.worldToVoxel(domain.localToWorld(np.array([0.5, 0.5]))))
domain_x = domain.rasterize( lambda pWS: pWS[0] )

domain2 = Domain2D(size, 512)
center_voxel2 = domain2.voxelToIndex(domain2.worldToVoxel(domain2.localToWorld(np.array([0.5, 0.5]))))
domain2_x = domain2.rasterize( lambda pWS: pWS[0] )
phi_groundtruth_mc_grid = np.loadtxt("anisotropic_absorption_2d/mc.txt")


plt.figure(figsize=(6, 6))
ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='both', direction='out')
ax.get_xaxis().tick_bottom()   # remove unneeded ticks 
ax.get_yaxis().tick_left()
plt.ylim([10.0e-2,10.0e1])
plt.loglog( domain2_x[center_voxel2[0], center_voxel2[1]:], phi_groundtruth_mc_grid[center_voxel2[0], center_voxel2[1]:], label="mc", color = 'r', linestyle=' ', marker='.', markersize=5 )
plt.loglog( domain_x[center_voxel[0], center_voxel[1]:], phi_grid[center_voxel[0], center_voxel[1]:], label="solution", color = 'g', linestyle=' ', marker='.', markersize=5 )
#plt.loglog( domain_x[center_voxel[0], center_voxel[1]:], phi_grid_iso[center_voxel[0], center_voxel[1]:], label="solution_iso", color = 'k', linestyle='-', markersize=5 )
plt.loglog( domain_x[center_voxel[0], center_voxel[1]:], phi_groundtruth_grid[center_voxel[0], center_voxel[1]:], label="groundtruth", color = 'g' )
plt.title("point light in vacuum (2d)")
plt.xlabel(r'$r \left[m\right]$', fontsize=18)
plt.ylabel(r'$\phi \left[\frac{W}{m}\right]$', fontsize=18)
plt.grid(True, linestyle='-',color='0.75')
plt.legend(loc='best')
plt.draw()
plt.show()

domain = Domain2D(1.0, 512)
pointlight1_center_ws = np.array([0.0, 0.0])
density = 5.0
sggx = SGGX2D( 0.01, 0.0, 1.0 )

phi_mc = np.loadtxt("anisotropic_absorption_2d/test.txt")

def compute_phi_groundtruth(pWS):
    ''' Compute groundtruth solution for single pointlight
    '''
    center = pointlight1_center_ws
    lightDir = center-pWS
    dist = np.linalg.norm(lightDir)
    lightDir = lightDir/dist
    power = 1.0

    radiance = power/(2.0*np.pi)

    L0 = radiance
    L0 *= geometry_term_2d(center, None, pWS, None)
    sigma_t = density*sggx.projectedArea(lightDir)
    L0 *= np.exp(-sigma_t*dist)
    
    return L0

phi_groundtruth = domain.rasterize(compute_phi_groundtruth)

fig = plt.figure(figsize=(11, 11))

plt.subplot(221)
plt.imshow(phi_mc, interpolation="nearest", extent = [domain.bound_min[0], domain.bound_max[0], domain.bound_min[1], domain.bound_max[1]])
plt.subplot(222)
plt.imshow(phi_groundtruth, interpolation="nearest", extent = [domain.bound_min[0], domain.bound_max[0], domain.bound_min[1], domain.bound_max[1]])


#plt.imshow(phi, interpolation="nearest", extent = [domain.bound_min[0], domain.bound_max[0], domain.bound_min[1], domain.bound_max[1]])

plt.subplot(223)
# 1d slice plot
center_voxel = domain.voxelToIndex(domain.worldToVoxel((domain.bound_max+domain.bound_min)/2.0))
domain_x = domain.rasterize( lambda pWS: pWS[0] )

ax = plt.gca()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.tick_params(axis='both', direction='out')
ax.get_xaxis().tick_bottom()   # remove unneeded ticks 
ax.get_yaxis().tick_left()
plt.ylim([10.0e-2,10.0e1])
#plt.loglog( domain_x[center_voxel[0], center_voxel[1]:], phi[center_voxel[0], center_voxel[1]:], label="solution", color = 'g', linestyle=' ', marker='.', markersize=5 )
plt.loglog( domain_x[center_voxel[0], center_voxel[1]:], phi_mc[center_voxel[0], center_voxel[1]:], label="mc", color = 'r', linestyle=' ', marker='.', markersize=5 )
plt.loglog( domain_x[center_voxel[0], center_voxel[1]:], phi_groundtruth[center_voxel[0], center_voxel[1]:], label="groundtruth", color = 'g' )

plt.grid(True, linestyle='-',color='0.75')
plt.legend(loc='best')


plt.show()